# Working with functions in Azure OpenAI
This notebook shows how to use the Chat Completions API in combination with functions to extend the current capabilities of GPT models. GPT models, do not inherently support real-time interaction with external systems, databases, or files. However, functions can be used to do so.

Overview: <br>
`tools` (previously called `functions`) is an optional parameter in the Chat Completion API which can be used to provide function specifications. This allows models to generate function arguments for the specifications provided by the user. 

Note: The API will not execute any function calls. Executing function calls using the outputed argments must be done by developers. 

## Setup

In [ ]:
# if needed, install and/or upgrade to the latest version of the OpenAI Python library
# %pip install --upgrade openai

In [ ]:
import os
import json
from openai import AzureOpenAI

# Load config values
with open(r"config.json") as config_file:
    config_details = json.load(config_file)

client = AzureOpenAI(
    azure_endpoint=config_details["AZURE_OPENAI_ENDPOINT"],  # The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
    api_key=os.getenv("AZURE_OPENAI_KEY"),  # The API key for your Azure OpenAI resource.
    api_version=config_details["OPENAI_API_VERSION"],  # This version supports function calling
)

model_name = config_details["MODEL_NAME"] # You need to ensure the version of the model you are using supports the function calling feature

## 1.0 Test functions

This code calls the model with the user query and the set of functions defined in the functions parameter. The model then can choose if it calls a function. If a function is called, the content will be in a strigified JSON object. The function call that should be made and arguments are location in:  response[`choices`][0][`function_call`].

In [ ]:
def get_function_call(messages, tool_choice="auto"):
    # Define the functions to use
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        },
    ]

    # Call the model with the user query (messages) and the functions defined in the functions parameter
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        tools=tools,
        tool_choice=tool_choice,
    )

    return response

### Forcing the use of a specific function or no function
By changing the value of the `tool_choice` parameter you can allow the model to decide what function to use, force the model to use a specific function, or force the model to use no function.

In [ ]:
first_message = [
    {"role": "user", "content": "What's the weather like in San Francisco?"}
]
# 'auto' : Let the model decide what function to call
print("Let the model decide what function to call:")
print(get_function_call(first_message, "auto").choices[0].message)

# 'none' : Don't call any function
print("Don't call any function:")
print(get_function_call(first_message, "none").choices[0].message)

# force a specific function call
print("Force a specific function call:")
print(
    get_function_call(
        first_message,
        tool_choice={"type": "function", "function": {"name": "get_current_weather"}},
    )
    .choices[0]
    .message
)

## 2.0 Defining functions
Now that we know how to work with functions, let's define some functions in code so that we can walk through the process of using functions end to end.

### Function #1: Get current time

In [ ]:
import pytz
from datetime import datetime


def get_current_time(location):
    try:
        # Get the timezone for the city
        timezone = pytz.timezone(location)

        # Get the current time in the timezone
        now = datetime.now(timezone)
        current_time = now.strftime("%I:%M:%S %p")

        return current_time
    except:
        return "Sorry, I couldn't find the timezone for that location."

In [ ]:
get_current_time("America/New_York")

### Function #2: Get stock market data
For simplicity, we're just hard coding some stock market data but you could easily edit the code to call out to an API to retrieve real-time data.

In [ ]:
import pandas as pd
import json


def get_stock_market_data(index):
    available_indices = [
        "S&P 500",
        "NASDAQ Composite",
        "Dow Jones Industrial Average",
        "Financial Times Stock Exchange 100 Index",
    ]

    if index not in available_indices:
        return "Invalid index. Please choose from 'S&P 500', 'NASDAQ Composite', 'Dow Jones Industrial Average', 'Financial Times Stock Exchange 100 Index'."

    # Read the CSV file
    data = pd.read_csv("stock_data.csv")

    # Filter data for the given index
    data_filtered = data[data["Index"] == index]

    # Remove 'Index' column
    data_filtered = data_filtered.drop(columns=["Index"])

    # Convert the DataFrame into a dictionary
    hist_dict = data_filtered.to_dict()

    for key, value_dict in hist_dict.items():
        hist_dict[key] = {k: v for k, v in value_dict.items()}

    return json.dumps(hist_dict)

In [ ]:
print(get_stock_market_data("NASDAQ Composite"))

### Function #3: Calculator 

In [ ]:
import math


def calculator(num1, num2, operator):
    if operator == "+":
        return str(num1 + num2)
    elif operator == "-":
        return str(num1 - num2)
    elif operator == "*":
        return str(num1 * num2)
    elif operator == "/":
        return str(num1 / num2)
    elif operator == "**":
        return str(num1**num2)
    elif operator == "sqrt":
        return str(math.sqrt(num1))
    else:
        return "Invalid operator"

In [ ]:
print(calculator(5, 5, "+"))

## 3.0 Calling a function using GPT

Steps for Function Calling: 

1. Call the model with the user query and a set of functions defined in the functions parameter.
2. The model can choose to call a function; if so, the content will be a stringified JSON object adhering to your custom schema (note: the model may generate invalid JSON or hallucinate parameters).
3. Parse the string into JSON in your code, and call your function with the provided arguments if they exist.
4. Call the model again by appending the function response as a new message, and let the model summarize the results back to the user.

### 3.1 Describe the functions so that the model knows how to call them

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "Get the current time in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The location name. The pytz is used to get the timezone for that location. Location names should be in a format like America/New_York, Asia/Bangkok, Europe/London",
                    }
                },
                "required": ["location"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_stock_market_data",
            "description": "Get the stock market data for a given index",
            "parameters": {
                "type": "object",
                "properties": {
                    "index": {
                        "type": "string",
                        "enum": [
                            "S&P 500",
                            "NASDAQ Composite",
                            "Dow Jones Industrial Average",
                            "Financial Times Stock Exchange 100 Index",
                        ],
                    },
                },
                "required": ["index"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "calculator",
            "description": "A simple calculator used to perform basic arithmetic operations",
            "parameters": {
                "type": "object",
                "properties": {
                    "num1": {"type": "number"},
                    "num2": {"type": "number"},
                    "operator": {
                        "type": "string",
                        "enum": ["+", "-", "*", "/", "**", "sqrt"],
                    },
                },
                "required": ["num1", "num2", "operator"],
            },
        },
    },
]

available_functions = {
    "get_current_time": get_current_time,
    "get_stock_market_data": get_stock_market_data,
    "calculator": calculator,
}

### 3.2 Define a helper function to validate the function call
It's possible that the models could generate incorrect function calls so it's important to validate the calls. Here we define a simple helper function to validate the function call although you could apply more complex validation for your use case.

In [ ]:
import inspect


# helper method used to check if the correct arguments are provided to a function
def check_args(function, args):
    sig = inspect.signature(function)
    params = sig.parameters

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True

In [ ]:
def run_conversation(messages, tools, available_functions):
    # Step 1: send the conversation and available functions to GPT
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )

    response_message = response.choices[0].message

    # Step 2: check if GPT wanted to call a function
    if response_message.tool_calls:
        print("Recommended Function call:")
        print(response_message.tool_calls[0])
        print()

        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors

        function_name = response_message.tool_calls[0].function.name

        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        function_to_call = available_functions[function_name]

        # verify function has correct number of arguments
        function_args = json.loads(response_message.tool_calls[0].function.arguments)
        if check_args(function_to_call, function_args) is False:
            return "Invalid number of arguments for function: " + function_name
        function_response = function_to_call(**function_args)

        print("Output of function call:")
        print(function_response)
        print()

        # Step 4: send the info on the function call and function response to GPT

        # adding assistant response to messages
        messages.append(
            {
                "role": response_message.role,
                "function_call": {
                    "name": response_message.tool_calls[0].function.name,
                    "arguments": response_message.tool_calls[0].function.arguments,
                },
                "content": None,
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("Messages in second request:")
        for message in messages:
            print(message)
        print()

        second_response = client.chat.completions.create(
            messages=messages,
            model=model_name,
        )  # get a new response from GPT where it can see the function response

        return second_response

In [ ]:
messages = [{"role": "user", "content": "What time is it in New York?"}]
assistant_response = run_conversation(messages, tools, available_functions)
print(assistant_response.choices[0].message)

### 4.0 Calling multiple functions together
In some cases, you may want to string together multiple function calls to get the desired result. We modified the `run_conversation()` function above to allow multiple function calls to be made.

In [ ]:
def run_multiturn_conversation(messages, tools, available_functions):
    # Step 1: send the conversation and available functions to GPT
    response = client.chat.completions.create(
        messages=messages,
        tools=tools,
        tool_choice="auto",
        model=model_name,
        temperature=0,
    )

    # Step 2: check if GPT wanted to call a function
    while response.choices[0].finish_reason == "tool_calls":
        response_message = response.choices[0].message
        print("Recommended Function call:")
        print(response_message.tool_calls[0])
        print()

        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors

        function_name = response_message.tool_calls[0].function.name

        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        function_to_call = available_functions[function_name]

        # verify function has correct number of arguments
        function_args = json.loads(response_message.tool_calls[0].function.arguments)
        if check_args(function_to_call, function_args) is False:
            return "Invalid number of arguments for function: " + function_name
        function_response = function_to_call(**function_args)

        print("Output of function call:")
        print(function_response)
        print()

        # Step 4: send the info on the function call and function response to GPT

        # adding assistant response to messages
        messages.append(
            {
                "role": response_message.role,
                "function_call": {
                    "name": response_message.tool_calls[0].function.name,
                    "arguments": response_message.tool_calls[0].function.arguments,
                },
                "content": None,
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("Messages in next request:")
        for message in messages:
            print(message)
        print()

        response = client.chat.completions.create(
            messages=messages,
            tools=tools,
            tool_choice="auto",
            model=model_name,
            temperature=0,
        )  # get a new response from GPT where it can see the function response

    return response

In [17]:
# Can add system prompting to guide the model to call functions and perform in specific ways
next_messages = [
    {
        "role": "system",
        "content": "Assistant is a helpful assistant that helps users get answers to questions. Assistant has access to several tools and sometimes you may need to call multiple tools in sequence to get answers for your users.",
    }
]
next_messages.append(
    {
        "role": "user",
        "content": "How much did S&P 500 change between July 12 and July 13? Use the calculator.",
    }
)

assistant_response = run_multiturn_conversation(
    next_messages, tools, available_functions
)
print("Final Response:")
print(assistant_response.choices[0].message)
print("Conversation complete!")

Conversation complete!
